In [1]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())


import time
s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
executed in 1.00 seconds.


In [2]:
# import the tidy3d package and configure it with your API key
import numpy as np
import tidy3d as td
import tidy3d.web as web
# web.configure("YOUR API KEY") # if authentication needed, uncomment this line and paste your API key here

# set up global parameters of simulation ( speed of light / wavelength in micron )
freq0 = td.C_0 / 0.75

# create structure - a box centered at 0, 0, 0 with a size of 1.5 micron and permittivity of 2
square = td.Structure(
    geometry=td.Box(center=(0, 0, 0), size=(1.5, 1.5, 1.5)),
    medium=td.Medium(permittivity=2.0)
)

# create source - A point dipole source with frequency freq0 on the left side of the domain
source = td.PointDipole(
    center=(-1.5, 0, 0),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=freq0 / 10.0),
    polarization="Ey",
)

# create monitor - Measures electromagnetic fields within the entire domain at z=0
monitor = td.FieldMonitor(
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="fields",
    colocate=True,
)

Let's make a list of simulations with different variations:

In [3]:
simulations = list()
for wavelength_step_i in [15, 20, 25]:
    # Initialize simulation - Combine all objects together into a single specification to run
    sim = td.Simulation(
        size=(4, 3, 3),
        grid_spec=td.GridSpec.auto(min_steps_per_wvl=wavelength_step_i),
        structures=[square],
        sources=[source],
        monitors=[monitor],
        run_time=120/freq0,
    )
    simulations.append(sim)

print(f"simulation grid is shaped {sim.grid.num_cells} for {int(np.prod(sim.grid.num_cells)/1e6)} million cells.")

simulation grid is shaped [179, 147, 147] for 3 million cells.


## Deploying Simulations Synchronously

If we wanted to run things synchronously, we can just iterate over a `web.run` command:

We can also create a batch:

In [4]:
# TODO

## Deploying Simulations Asynchronously

Now, let's explore multiple ways of running this simulation asynchronously:

In [5]:
from tidy3d.web.api.asynchronous import run

In [6]:
[run(simulation_i, task_name="quickstart", path="data/data.hdf5", verbose=True) for simulation_i in simulations]

[<coroutine object run at 0x7aa7e6a805f0>,
 <coroutine object run at 0x7aa7e6a80660>,
 <coroutine object run at 0x7aa7e6a80740>]

In [7]:
async def main():
    await asyncio.gather(*[run(simulation_i, task_name="quickstart", path="data/data.hdf5", verbose=True) for simulation_i in simulations])

await main()

12:39:59 CEST Created task 'quickstart' with task_id                            
              'fdve-f4af29a5-1a3a-4626-b0bc-10e5c95f6213' and task_type 'FDTD'.

              View task using web UI at                                         
              ]8;id=69604;https://tidy3d.simulation.cloud/workbench?taskId=fdve-f4af29a5-1a3a-4626-b0bc-10e5c95f6213\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=514624;https://tidy3d.simulation.cloud/workbench?taskId=fdve-f4af29a5-1a3a-4626-b0bc-10e5c95f6213\taskId]8;;\]8;id=69604;https://tidy3d.simulation.cloud/workbench?taskId=fdve-f4af29a5-1a3a-4626-b0bc-10e5c95f6213\=]8;;\]8;id=534519;https://tidy3d.simulation.cloud/workbench?taskId=fdve-f4af29a5-1a3a-4626-b0bc-10e5c95f6213\fdve]8;;\]8;id=69604;https://tidy3d.simulation.cloud/workbench?taskId=fdve-f4af29a5-1a3a-4626-b0bc-10e5c95f6213\-f4af29a5-1a]8;;\
              ]8;id=69604;https://tidy3d.simulation.cloud/workbench?taskId=fdve-f4af29a5-1a3a-4626-b0bc-10e5c95f6213\3a-4626-b0bc-10e5c95f6213']8;;\.

Output()

12:40:03 CEST Unable to download 'output/simulation_data.hdf5.gz'.

12:40:04 CEST ERROR: Failed to download the simulation data file from the       
              server. Please confirm that the task was successfully run.        

...

              View task using web UI at                                         
              ]8;id=958173;https://tidy3d.simulation.cloud/workbench?taskId=fdve-79afde5a-e15b-462f-ae67-1f69dc6e7276\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=158876;https://tidy3d.simulation.cloud/workbench?taskId=fdve-79afde5a-e15b-462f-ae67-1f69dc6e7276\taskId]8;;\]8;id=958173;https://tidy3d.simulation.cloud/workbench?taskId=fdve-79afde5a-e15b-462f-ae67-1f69dc6e7276\=]8;;\]8;id=840917;https://tidy3d.simulation.cloud/workbench?taskId=fdve-79afde5a-e15b-462f-ae67-1f69dc6e7276\fdve]8;;\]8;id=958173;https://tidy3d.simulation.cloud/workbench?taskId=fdve-79afde5a-e15b-462f-ae67-1f69dc6e7276\-79afde5a-e1]8;;\
              ]8;id=958173;https://tidy3d.simulation.cloud/workbench?taskId=fdve-79afde5a-e15b-462f-ae67-1f69dc6e7276\5b-462f-ae67-1f69dc6e7276']8;;\.

Output()

12:40:13 CEST Unable to download 'output/simulation_data.hdf5.gz'.

12:40:14 CEST ERROR: Failed to download the simulation data file from the       
              server. Please confirm that the task was successfully run.        

WebError: Failed to download the simulation data file from the server. Please confirm that the task was successfully run.